In [116]:
import pandas as pd
import geopandas as gpd

In [117]:
data = pd.read_parquet('../data/raw_data_delays.parquet')
stations_gps = gpd.read_file('../data/spatial_data/stations_gps.shp', encoding='utf-8')

In [118]:
routes_stations = data[['Relacja', 'Stacja']].groupby('Relacja')['Stacja'].agg(list).reset_index()
routes_stations['Stacja'] = routes_stations['Stacja'].apply(lambda x: ','.join(map(str, x)))
routes_stations['Stacja'] = routes_stations['Stacja'].str.split(',')
routes_stations = routes_stations.explode('Stacja').drop_duplicates()
routes_stations = routes_stations.merge(stations_gps, how='left', on='Stacja')

In [120]:
routes_stations['station_count_on_curr_station'] = routes_stations.groupby(['Relacja']).cumcount()
routes_stations['full_route_station_count'] = data.groupby(['Relacja'])['Relacja'].transform('count')

In [124]:
routes = routes_stations['Relacja'].unique()
routes_list_out = []
for route in routes:
    temp_df = routes_stations[routes_stations['Relacja']==route].copy().reset_index(drop=True)
    temp_df['station_count_on_curr_station'] = temp_df.groupby(['Relacja']).cumcount()
    temp_df['full_route_station_count'] = temp_df.groupby(['Relacja'])['Relacja'].transform('count')

    routes_list_out.append(temp_df)

In [127]:
routes_list_out[500]

,Relacja,Stacja,geometry,station_count_on_curr_station,full_route_station_count
0,Lubliniec - Wrocław Główny,Lubliniec,POINT (18.69003 50.67318),0,27
1,Lubliniec - Wrocław Główny,Kochcice-Glinica,POINT (18.65720 50.71472),1,27
2,Lubliniec - Wrocław Główny,Ciasna,POINT (18.62942 50.75635),2,27
3,Lubliniec - Wrocław Główny,Sieraków Śląski,POINT (18.59360 50.79882),3,27
4,Lubliniec - Wrocław Główny,Sowczyce,POINT (18.50167 50.84590),4,27
5,Lubliniec - Wrocław Główny,Olesno Śląskie,POINT (18.42483 50.88207),5,27
6,Lubliniec - Wrocław Główny,Stare Olesno,POINT (18.37495 50.91553),6,27
7,Lubliniec - Wrocław Główny,Bąków,POINT (18.31458 50.95612),7,27
8,Lubliniec - Wrocław Główny,Kluczbork,POINT (18.20101 50.97108),8,27
9,Lubliniec - Wrocław Główny,Smardy,POINT (18.14432 50.99224),9,27


In [50]:
routes = data['Relacja'].drop_duplicates()\
    .str.split(' - ')\
    .apply(lambda x: sorted(x))\
    .apply(lambda x: ','.join(map(str,x)))\
    .drop_duplicates()\
    .reset_index(drop=True)\
    .to_frame()

In [51]:
routes['Station_A'] = routes['Relacja'].str.split(',').str[0]
routes['Station_B'] = routes['Relacja'].str.split(',').str[1]

In [52]:
routes = routes\
    .merge(stations_gps, how='left', left_on='Station_A', right_on='Stacja')\
    .merge(stations_gps, how='left', left_on='Station_B', right_on='Stacja')\
    .drop(['Stacja_x','Stacja_y'], axis=1)\
    .rename(columns={'geometry_x':'geometry_A', 'geometry_y':'geometry_B'})

In [53]:
geoms = ['geometry_A','geometry_B']
suffixes = ['A', 'B']

for suffix, geom in zip(suffixes, geoms):
    print(suffix)
    print(geom)

# routes['latA'] = routes['geometry_A'].apply(lambda point: point.y)
# routes['lonA'] = routes['geometry_A'].apply(lambda point: point.x)
# routes['latB'] = routes['geometry_B'].apply(lambda point: point.y)
# routes['lonB'] = routes['geometry_B'].apply(lambda point: point.x)

A
geometry_A
B
geometry_B


In [54]:
routes.drop(columns=['geometry_A','geometry_B'],inplace=True)

In [16]:
routes.to_parquet('../data/routes_empty.parquet', index=False)

,Relacja,Station_A,Station_B,latA,lonA,latB,lonB
0,"Przeworsk,Rzeszów Główny",Przeworsk,Rzeszów Główny,50.067161,22.503232,50.043213,22.005808
1,"Lublin Główny,Zamość Wschód",Lublin Główny,Zamość Wschód,51.231485,22.568419,50.722481,23.275995
2,"Katowice,Zwardoń",Katowice,Zwardoń,50.257607,19.017146,49.504320,18.977420
3,"Ostrów Wielkopolski,Wrocław Główny",Ostrów Wielkopolski,Wrocław Główny,51.649247,17.805625,51.097968,17.037530
4,"Gryfino,Szczecin Główny",Gryfino,Szczecin Główny,53.254119,14.492962,53.418241,14.549407
...,...,...,...,...,...,...,...
709,"Wrocław Główny,Ziębice",Wrocław Główny,Ziębice,51.097968,17.037530,50.608500,17.033886
710,"Szczecin Główny,Łobez",Szczecin Główny,Łobez,53.418241,14.549407,53.636654,15.627729
711,"Warszawa Wschodnia,Łódź Kaliska",Warszawa Wschodnia,Łódź Kaliska,52.251573,21.052298,51.757822,19.430542
712,"Nowa Sól,Poznań Główny",Nowa Sól,Poznań Główny,51.798960,15.708799,52.401627,16.911579


In [17]:
routes[['latA','lonA','latB','lonB']].to_csv('../data/coordinates_to_js.csv',index=False)

,latA,lonA,latB,lonB
0,50.067161,22.503232,50.043213,22.005808
1,51.231485,22.568419,50.722481,23.275995
2,50.257607,19.017146,49.504320,18.977420
3,51.649247,17.805625,51.097968,17.037530
4,53.254119,14.492962,53.418241,14.549407
...,...,...,...,...
709,51.097968,17.037530,50.608500,17.033886
710,53.418241,14.549407,53.636654,15.627729
711,52.251573,21.052298,51.757822,19.430542
712,51.798960,15.708799,52.401627,16.911579


In [55]:
routes

,Relacja,Station_A,Station_B
0,"Przeworsk,Rzeszów Główny",Przeworsk,Rzeszów Główny
1,"Lublin Główny,Zamość Wschód",Lublin Główny,Zamość Wschód
2,"Katowice,Zwardoń",Katowice,Zwardoń
3,"Ostrów Wielkopolski,Wrocław Główny",Ostrów Wielkopolski,Wrocław Główny
4,"Gryfino,Szczecin Główny",Gryfino,Szczecin Główny
...,...,...,...
709,"Wrocław Główny,Ziębice",Wrocław Główny,Ziębice
710,"Szczecin Główny,Łobez",Szczecin Główny,Łobez
711,"Warszawa Wschodnia,Łódź Kaliska",Warszawa Wschodnia,Łódź Kaliska
712,"Nowa Sól,Poznań Główny",Nowa Sól,Poznań Główny


In [94]:
to_group = data[['Relacja','Stacja']].drop_duplicates()
grouped_df = to_group.groupby('Relacja')['Stacja'].apply(lambda x: sorted(x)).reset_index()
grouped_df['Stacja'] = grouped_df['Stacja'].apply(lambda x: ','.join(map(str, x)))

In [95]:
grouped_df['Relacja'] = grouped_df['Relacja'].str.split(' - ').apply(lambda x: sorted(x)).apply(lambda x: ','.join(map(str,x)))

In [96]:
grouped_df = grouped_df.drop_duplicates().reset_index(drop=True)

In [97]:
grouped_df = grouped_df[['Relacja']].drop_duplicates().reset_index(drop=True)

In [98]:
grouped_df['Station_A'] = grouped_df['Relacja'].str.split(',').str[0]
grouped_df['Station_B'] = grouped_df['Relacja'].str.split(',').str[1]

In [103]:
grouped_df = \
pd.merge(grouped_df, stations_gps, how='left', left_on='Station_A', right_on='Stacja')\
    .merge(stations_gps, how='left', left_on='Station_B', right_on='Stacja')\
    .drop(['Stacja_x','Stacja_y'], axis=1)\
    .rename(columns={'geometry_x':'geometry_A', 'geometry_y':'geometry_B'})

In [104]:
grouped_df

,Relacja,Station_A,Station_B,geometry_A,geometry_B
0,"Augustów,Białystok",Augustów,Białystok,POINT (23.02645 53.85254),POINT (23.13538 53.13364)
1,"Białogard,Kołobrzeg",Białogard,Kołobrzeg,POINT (15.97765 54.00999),POINT (15.57090 54.18219)
2,"Białystok,Czeremcha",Białystok,Czeremcha,POINT (23.13538 53.13364),POINT (23.35351 52.51875)
3,"Białystok,Częstochowa Stradom",Białystok,Częstochowa Stradom,POINT (23.13538 53.13364),POINT (19.10700 50.79732)
4,"Białystok,Ełk",Białystok,Ełk,POINT (23.13538 53.13364),POINT (22.36200 53.82560)
...,...,...,...,...,...
709,"Gniezno,Środa Wielkopolska",Gniezno,Środa Wielkopolska,POINT (17.60323 52.52967),POINT (17.27419 52.21833)
710,"Szczecin Główny,Świnoujście",Szczecin Główny,Świnoujście,POINT (14.54941 53.41824),POINT (14.26673 53.90472)
711,"Kraków Główny,Żegiestów",Kraków Główny,Żegiestów,POINT (19.94780 50.06867),POINT (20.78526 49.36756)
712,"Siedlce,Żyrardów",Siedlce,Żyrardów,POINT (22.27157 52.16196),POINT (20.44898 52.05247)
